## Init

In [ ]:
from dataclasses import dataclass

import numpy as np
import pandas as pd

import matplotlib.pylab as plt
import seaborn as sns

from scipy.optimize import minimize_scalar
from scipy.sparse.linalg import eigsh

from luescher_nd.hamiltonians.longrange import PhenomLRHamiltonian
from luescher_nd.zeta.zeta3d import Zeta3D
from luescher_nd.zeta.extern.pyzeta import zeta

In [ ]:
%load_ext blackcellmagic

In [ ]:
HBARC = 197

mpi = 134 / HBARC

mpi

In [ ]:
def gp(p, gbar=0.8945, M=20):
    return gbar * np.sqrt(8 * np.pi) * M**3 / (p**2 + M**2)**2

In [ ]:
e0 = -2.225

h = PhenomLRHamiltonian(n1d=20, epsilon=0.25, nstep=None, M=mpi*10)

ee = eigsh(h.op, which="SA", return_eigenvectors=False, k=1)

@dataclass
class Kernel:
    n1d: int
    epsilon: float
    nstep: bool
    M: float
    e0: float
        
    def res(self, gbar):
        h = PhenomLRHamiltonian(n1d=self.n1d, epsilon=self.epsilon, nstep=self.nstep, M=self.M, gbar=gbar)
        ee = eigsh(h.op, which="SA", return_eigenvectors=False, k=1)
        return (e0 - ee.min())**2


kernel = Kernel(10, 0.25, None, 10.0, -2.225)

res = minimize_scalar(kernel.res, bracket=(1.e-4, 1.e2))

res.x

In [ ]:
p = np.linspace(0, 20, 500)

h = PhenomLRHamiltonian(n1d=20, epsilon=0.25, nstep=None, M=mpi*10)

fig, ax = plt.subplots(dpi=250)

ax.plot(p, gp(p, gbar=h.gbar, M=h.M))

ax.plot(np.sqrt(h.p2), h._gp, ".")

ax.set_yscale("log")

plt.show(fig)

In [ ]:
def pcotd(p, gbar, mu, M):
    d0 = 16 * gbar ** 2 * mu
    res = -(5 * gbar ** 2 * mu * M - 4 * M ** 2) / d0
    res += (15 * gbar ** 2 * mu + 16 * M) / d0 / M * p ** 2
    res += (5 * gbar ** 2 * mu + 24 * M) / d0 / M ** 3 * p ** 4
    res += (gbar ** 2 * mu + 16 * M) / d0 / M ** 5 * p ** 6
    res += 4 * M / d0 / M ** 7 * p ** 8
    return res

## Finite Volume g

In [ ]:
def gbarfix(gamma, M, mu):
    return (
        2
        * (gamma + M) ** 2
        / np.sqrt(mu * M ** 3 * (gamma ** 2 + 5 * M ** 2 + 4 * gamma * M))
        * M
    )

In [ ]:
L = 5.0
e0 = -2.225
M = 10.0
nstep = None

data = []

for n1d in [10, 15, 20, 25]:#, 30, 35]:  # , 40, 45, 50]:
    print(n1d)
    epsilon = L / n1d

    kernel = Kernel(n1d, epsilon=epsilon, nstep=nstep, M=M, e0=e0)
    res = minimize_scalar(kernel.res, bracket=(1.0e-4, 1.0e2))

    h = PhenomLRHamiltonian(n1d=n1d, epsilon=epsilon, nstep=None, M=M, gbar=res.x)

    ee = eigsh(h.op, which="SA", return_eigenvectors=False, k=20)

    mu = h.m / 2
    p2 = 2 * mu * ee
    x = (h.L / 2 / np.pi) ** 2 * p2

    z = Zeta3D(L, h.epsilon, h.nstep)(x)

    for xi, zi in zip(x, z):
        data.append(
            {
                "x": xi,
                "y": zi / np.pi / h.L,
                "epsilon": h.epsilon,
                "L": h.L,
                "n1d": n1d,
                "gbar": h.gbar,
            }
        )

df = pd.DataFrame(data).dropna().drop_duplicates()
df["y_old"] = zeta(df["x"])
df.head()


In [ ]:
fig, ax = plt.subplots(dpi=250)

p = np.linspace(-np.sqrt(abs(p2.min())), np.sqrt(p2.max()), 1000)
xp = (h.L / 2 / np.pi) ** 2 * p ** 2 * np.sign(p)

g0 = np.sqrt(-2 * mu * e0)
gbar0 = gbarfix(g0, M, mu)

sns.scatterplot(x="x", y="y", data=df, ax=ax, style="n1d", color="green", s=50, lw=2)
sns.scatterplot(x="x", y="y_old", data=df, ax=ax, style="n1d", color="red", legend=False)

ax.plot(
    xp,
    pcotd(p * (-1j) ** (p < 0), gbar0, h.m / 2, h.M).real,
    label="Analytic",
    ls="--",
    color="black",
    zorder=-1,
    lw=1
)

ax.plot(np.nan, np.nan, color="red", label="Original Lüscher")
ax.plot(np.nan, np.nan, color="green", label="Dispersion Lüscher")

ax.legend(fontsize=8, loc="center left", frameon=False, )

ax.set_ylabel("Effective range expansion\n" r"$p \cot(\delta_0(p))$ [fm$^{-1}$]")
ax.set_xlabel(
    "Normalized scattering momentum\n" r"$x = p^2 \left(\frac{L}{2\pi}\right)^2$"
)

ax.set_title(r"Potential fitted to reproduce infinite volume continuum $E_0 = -2.225$ MeV", size=8)




plt.show(fig)


## Infinite Volume g

In [ ]:
L = 5.0
e0 = -2.225
M = 10.0
nstep = None

data = []

for n1d in [10, 15, 20, 25]:#, 30, 35]:  # , 40, 45, 50]:
    print(n1d)
    epsilon = L / n1d

    h = PhenomLRHamiltonian(n1d=n1d, epsilon=epsilon, nstep=None, M=M, gbar=gbar0)

    ee = eigsh(h.op, which="SA", return_eigenvectors=False, k=20)

    mu = h.m / 2
    p2 = 2 * mu * ee
    x = (h.L / 2 / np.pi) ** 2 * p2

    z = Zeta3D(L, h.epsilon, h.nstep)(x)

    for xi, zi in zip(x, z):
        data.append(
            {
                "x": xi,
                "y": zi / np.pi / h.L,
                "epsilon": h.epsilon,
                "L": h.L,
                "n1d": n1d,
                "gbar": h.gbar,
            }
        )

df2 = pd.DataFrame(data).dropna().drop_duplicates()
df2["y_old"] = zeta(df2["x"])
df2.head()



In [ ]:
fig, ax = plt.subplots(dpi=250)

p = np.linspace(-np.sqrt(abs(p2.min())), np.sqrt(p2.max()), 1000)
xp = (h.L / 2 / np.pi) ** 2 * p ** 2 * np.sign(p)

g0 = np.sqrt(-2 * mu * e0)
gbar0 = gbarfix(g0, M, mu)

sns.scatterplot(x="x", y="y", data=df2, ax=ax, style="n1d", color="green", s=50, lw=2)
sns.scatterplot(x="x", y="y_old", data=df2, ax=ax, style="n1d", color="red", legend=False)

ax.plot(
    xp,
    pcotd(p * (-1j) ** (p < 0), gbar0, h.m / 2, h.M).real,
    label="Analytic",
    ls="--",
    color="black",
    zorder=-1,
    lw=1
)

ax.plot(np.nan, np.nan, color="red", label="Original Lüscher")
ax.plot(np.nan, np.nan, color="green", label="Dispersion Lüscher")

ax.legend(fontsize=8, loc="center left", frameon=False, )

ax.set_ylabel("Effective range expansion\n" r"$p \cot(\delta_0(p))$ [fm$^{-1}$]")
ax.set_xlabel(
    "Normalized scattering momentum\n" r"$x = p^2 \left(\frac{L}{2\pi}\right)^2$"
)

ax.set_title(r"Potential uses infinite volume continuum parameters", size=8)

plt.show(fig)


## Finite Volume g